In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader

db = FAISS.from_documents(
    PyMuPDFLoader("./2312.10997v5.pdf").load_and_split(),
    HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2"
    )
)

In [6]:
db.index.ntotal

38

In [10]:
db.similarity_search_with_relevance_scores(
    "what are the different types of retrieval augmentation processes"
)

[(Document(metadata={'source': './2312.10997v5.pdf', 'file_path': './2312.10997v5.pdf', 'page': 0, 'total_pages': 21, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.25', 'creationDate': 'D:20240328005445Z', 'modDate': 'D:20240328005445Z', 'trapped': ''}, page_content='fine-tuning techniques.\nThe burgeoning field of RAG has experienced swift growth,\nyet it has not been accompanied by a systematic synthesis that\ncould clarify its broader trajectory. This survey endeavors to\nfill this gap by mapping out the RAG process and charting\nits evolution and anticipated future paths, with a focus on the\nintegration of RAG within LLMs. This paper considers both\ntechnical paradigms and research methods, summarizing three\nmain research paradigms from over 100 RAG studies, and\nanalyzing key technologies in the core stages of “Retrieval,”\n“Generation,” and “Augmentation.” On the other hand, current\nres

In [16]:
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0, model_name="gemma2-9b-it")

In [17]:
from langchain_core.prompts import ChatPromptTemplate

system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
for chunk in chain.stream({"text": "what are the different types of retrieval augmentation processes"}):
    print(chunk.content, end="", flush=True)

Retrieval augmentation is a powerful technique used in information retrieval to improve the accuracy and relevance of search results. It involves enriching the original query with additional information to guide the retrieval process more effectively. 

Here are some common types of retrieval augmentation processes:

**1. Query Expansion:**

* **Synonym Replacement:** Replacing words in the query with their synonyms to capture different meanings and related concepts.
* **Word Sense Disambiguation:** Identifying the intended meaning of ambiguous words in the query and using the appropriate sense for retrieval.
* **Concept Expansion:** Adding related concepts or terms to the query based on semantic relationships, ontologies, or knowledge graphs.
* **Negation Handling:** Identifying and incorporating negation terms (e.g., "not," "without") to refine the search scope.

**2. Document Augmentation:**

* **Passage Retrieval:** Instead of retrieving entire documents, focus on finding relevant 

In [18]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": db.as_retriever() | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | chat
    | StrOutputParser()
)

rag_chain.invoke("what are the different types of retrieval augmentation processes")

'There are three main types of retrieval augmentation processes: iterative retrieval, recursive retrieval, and adaptive retrieval.  Thanks for asking! \n\n\n'